# 18650 Battery Life: Survival Analysis vs Machine Learning

### Aryan Bhardwaj, Cormac Dacker, Tyler Gomez Riddick, Avery Pike

## Data Preprocessing

### Data Loading

The data for this project are the entire life-cycle of 26 battery packs consisting of two 18650 battery, which are rechargeable lithium-ion batteries. Each of the 26 battery packs has its own dedicated CSV, recording data about the battery once per second during the charging, rest, and discharging phases. Because there are only 26 battery packs in these data, we decided to focus on the discharge phases only, as each battery pack has several discharge phases through it's lifecycle. First though, we have to load in the data, and there is a lot of it (a few million rows per csv on average).

Because all of the battery data were stored in separate CSVs, we first needed to combine each group (regular, second life, recommissioned) into their own CSV consisting of all of the battery packs in that dataset.

In [2]:
import os
import glob  # pip install glob2
import pandas as pd
import warnings
import time
import tqdm  # pip install tqdm
import numpy as np


In [3]:
print(os.getcwd())

/Users/leotyler/venv/GitHub/18650-Accelerated-Battery-Life-Testing


In [17]:
csv_paths = ['/battery_alt_dataset/regular_alt_batteries',
             #'/battery_alt_dataset/second_life_batteries',
             #'/battery_alt_dataset/recommissioned_batteries'
]

#root = os.path.dirname(os.path.abspath('csv_combine.py'))
root = os.getcwd()
print(root)


def concatinateCSVs(folderPath, ignore_list=[]):  # combines all csv files in a folder into df
    # print('concatinating csvs in', folderPath)
    #root = os.path.dirname(os.path.realpath('csv_combine.py'))
    print(root)
    warnings.filterwarnings("ignore")
    os.chdir(folderPath)
    # all the filenames with a .csv format
    allFilenames = [i for i in glob.glob("*.{}".format("csv"))]
    combinedFilesData = []
    for file in tqdm.tqdm(allFilenames, desc='Combining ' + folderPath.split('/')[-1]):
        if file in ignore_list:
            continue
        try:
            df = pd.read_csv(file)
            # # drop all rows except the one with the greatest time value
            # df = df.drop_duplicates(subset=['time'], keep='last')
            combinedFilesData.append(df)
        except pd.errors.EmptyDataError:
            print(file, "is empty")
            # os.remove(f)
            # print(f, "has been deleted")
            continue
        except pd.errors.ParserError:
            print(file, "parse error")
            continue
    try:  # concatinate all csv data in a folder
        combinedFilesData = pd.concat([file for file in combinedFilesData], )
    except ValueError:
        combinedFilesData = []
    os.chdir(root)
    return pd.DataFrame(combinedFilesData)


def combine_csvs(csv_paths):  # builds the one files from folders
    for csv_path in csv_paths:
        (concatinateCSVs(root + csv_path,
                         ignore_list=['battery40.csv', 'battery41.csv',
                                      'battery50.csv', 'battery51.csv']).to_csv(root + csv_path + '.csv', index=False))

        # * 9.30A: Battery pack 0.1 and 1.1
        # * 12.9A: Battery pack 3.1 and 2.2
        # * 14.3A: Battery pack 2.3 and 5.2
        # * 16.0A: Battery pack 0.0 and 1.0
        # * 19.0A: Battery pack 2.0, 3.0 and 2.1


        return


def convert_to_preferred_format(sec):  # only to be fancy
    sec = sec % (24 * 3600)
    sec %= 3600
    min = sec // 60
    sec %= 60
    # print("seconds value in minutes:",min)
    return "%02d:%02d" % (min, sec)


if __name__ == '__main__':
    start_time = time.time()
    combine_csvs(csv_paths)
    print('Time taken:', convert_to_preferred_format(time.time() - start_time))


/Users/leotyler/venv/GitHub/18650-Accelerated-Battery-Life-Testing
/Users/leotyler/venv/GitHub/18650-Accelerated-Battery-Life-Testing


Combining regular_alt_batteries: 100%|██████████| 15/15 [00:07<00:00,  2.12it/s]


Time taken: 01:01


### Data Combining: Reducing Dischrge Phases

Because we are focusing on only the discharge phases, we want to break each of the battery datasets into their discharge phases, and then find the average voltage, temperature, and current for each discharge cycle. This will reduce each discharge phase down to a single row instead of the millions we have. To start, we have to load in the new datasets and process them.

In [20]:
# Numbering each distinct discharge phase
def battLabeller(df):
    '''
    df - battery dataset
    feed battery dataset, add column to denote discharge phase
    '''
    label = 1
    df['dischargePhase'] = 0
    # for time in range(len(df['start_time'])-1):
    for time in tqdm.tqdm(range(len(df['start_time'])-1), total=len(df['start_time'])-1):
        if df['mode'][time] == -1:
            df.at[time, 'dischargePhase'] = label
            if df['mode'][time+1] == 0 or df['mode'][time+1] == 1:
                label += 1
    return df

In [21]:
# This function calculates the time it takes the battery pack in a phase to
# discharge its charge
def timeFinder(df):
    '''
    df - dataframe
    takes dataframe and finds time from start of phase to end of phase
    '''
    df['startTime'] = 0
    df['finishTime'] = 0
    #for phase in df['dischargePhase'].unique():
    for phase in tqdm.tqdm(df['dischargePhase'].unique(), total=len(df['dischargePhase'].unique())):
        currPhase = df[df['dischargePhase'] == phase]
        #print(currPhase)
        #print(phase)
        # Find start and end time
        start = currPhase['time'].iloc[0] 
        finish = currPhase['time'].iloc[-1]
        
        # Update start and end time
        df.loc[df['dischargePhase'] == phase, 'startTime'] = start
        df.loc[df['dischargePhase'] == phase, 'finishTime'] = finish
    return df

In [22]:
def process_battery_data(df):
    '''
    Takes in df of battery data and reduces each phase
    to a single row, averaging temp, voltage, and current
    and finding the time to death for each discharge phase
    
    Parameters:
    df (dataFrame): input df of battery data sent through
    battLabeller and timeFinder
    
    Returns:
    df (transformed): averaged values and new column
    end_time which is the difference of finishTime and startTime
    '''
    # Run battery df through functions time_end via difference
    df = battLabeller(df)
    df = df[df['mode'] != 0]
    df = df[df['mode'] != 1]
    df = timeFinder(df)
    cols = ['voltage_charger','temperature_battery','current_load','dischargePhase','startTime','finishTime']
    df = df[cols]
    df = df.dropna(subset=['dischargePhase'])
    
    df['time_end'] = (df['finishTime'] - df['startTime'])
    
    # Cast columns to floats
    numeric_cols = ['voltage_charger','temperature_battery','current_load']
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col],errors = 'coerce')
    
    # Group by dischargePhase and aggregate other columns
    aggregated_df = df.groupby('dischargePhase').agg({
        'voltage_charger':'mean',
        'temperature_battery':'mean',
        'current_load':'mean',
        'time_end':'first'
    }).reset_index()
    aggregated_df['mode'] = 1
    
    return aggregated_df

In [24]:
# Read in combined csvs
regular = pd.read_csv('battery_alt_dataset/regular_alt_batteries.csv')
second = pd.read_csv('battery_alt_dataset/second_life_batteries.csv')
recommissioned = pd.read_csv('battery_alt_dataset/recommissioned_batteries.csv')

# Run through the battery data processor
regular = process_battery_data(regular)
second = process_battery_data(second)
recommissioned = process_battery_data(recommissioned)

# Write to new csvs
regular.to_csv('regular.csv')
second.to_csv('second.csv')
recommissioned.to_csv('recommissioned.csv')

100%|██████████| 3626/3626 [00:15<00:00, 234.41it/s]


# Survival Analysis and Weibull Regression

All of the survival analysis done on these data was performed in R, due to our group's familiarity with working in the language with survival analysis. Thus, the next sections of this notebook will not be able to render the work we did, but the R markdowns will be included as part of the submission.

```{r}
library(fitdistrplus)
library(survminer)
library(tidyverse)
library(ggplot2)
library(survival)
library(skimr)

regular = read_csv('regular.csv')
recom = read_csv('recommissioned.csv')
second = read_csv('second.csv')
```

Filtering out times shorter than 10 seconds or NaN values, and temperatures measured at 0. Calculating the Arrhenius temperature and the log voltage for use in the survival analysis models. Additionally, adding in a column for failure (needed for the survival analysis). Assume everywhere "data" occurs that it is filled with one of the three battery DataFrames.

```{r}
# filter out temps < 0
data = data %>% filter(temperature_battery > 0)

# drop times at 0
data = data %>% filter(time_end > 10)

# add a col for failure (they all failed)
data$failure = 1

# drop instances where time_end is NA or 0
data = data %>% filter(!is.na(time_end) & time_end != 0)
```

Performing the Weibull regression using the current_load, log_volt, and arr_temp columns. No interaction terms. Values shown below are for the regular batteries (regular.csv)

```{r}
weibull_fit = survreg(Surv(time_end, failure) ~ current_load +
  log_volt +
  arr_temp
                      , data = data, dist = "weibull")
summary(weibull_fit)
```
```{r}
                        Value Std. Error       z       p
(Intercept)          2.640372   0.140380   18.81 < 2e-16
current_load        -0.081932   0.001081  -75.79 < 2e-16
voltage_charger      0.780425   0.020004   39.01 < 2e-16
temperature_battery -0.001641   0.000416   -3.95 7.9e-05
Log(scale)          -1.754017   0.011486 -152.71 < 2e-16

Scale= 0.173 

Weibull distribution
Loglik(model)= -23355.7   Loglik(intercept only)= -27754.5
	Chisq= 8797.56 on 3 degrees of freedom, p= 0 
Number of Newton-Raphson Iterations: 9 
n= 3743 
```

Next, computing the relevant model metrics: mean squared error (MSE), Akaike information criterion (AIC), and R-squared. Below are the results for the regular batteries. The submitted R-markdown file will show the results for all three datasets.

```{r}
model_metrics = function(model, regular) {
  pred = predict(model, type = "response")
  mse = mean((pred - regular$time_end)^2)
  aic = AIC(model)
  r2 = 1 - mse / var(regular$time_end)
  print("MSE:")
  print(mse)
  print("AIC:")
  print(aic)
  print("R^2:")
  print(r2)
  return(c(mse, aic, r2))
}
model_metrics(weibull_fit, regular)

[1] "MSE:"
[1] 42707.18
[1] "AIC:"
[1] 46756.98
[1] "R^2:"
[1] 0.8550565
[1] 4.270718e+04 4.675698e+04 8.550565e-01
```

# Maching Learning Models

We focused on three models of increasing complexity: linear regression, random forest, and neural networks. We used the classifiers from sklearn for linear regression and random forest, and tensorflow for the neural network. For each of these, we wanted to find the $MSE$, $AIC$, and $R^2$ terms. Below is the code for each model.

In [30]:
regular = pd.read_csv('regular.csv')
recomissioned = pd.read_csv('recommissioned.csv')
second = pd.read_csv('second.csv')

Below is some cleaning code.

### Cleaning - Regular

In [50]:
# Adding in the calculated Arrhenius temperature and log voltage columns

regular['arrhenius_temperature'] = 11605 / (regular['temperature_battery'] + 273.15)

regular['log_voltage'] = np.log10(regular['voltage_charger'])

# Filtering temperatures above 0 and ending times greater than 10 seconds

regular = regular[regular['temperature_battery'] > 0]

regular = regular[regular['time_end'] > 10]

regular['temp_volt'] = regular['arrhenius_temperature'] * regular['log_voltage']

regular.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3743 entries, 0 to 3825
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             3743 non-null   int64  
 1   dischargePhase         3743 non-null   int64  
 2   voltage_charger        3743 non-null   float64
 3   temperature_battery    3743 non-null   float64
 4   current_load           3743 non-null   float64
 5   time_end               3743 non-null   float64
 6   mode                   3743 non-null   int64  
 7   arrhenius_temperature  3743 non-null   float64
 8   log_voltage            3743 non-null   float64
 9   temp_volt              3743 non-null   float64
dtypes: float64(7), int64(3)
memory usage: 321.7 KB


In [32]:
regular.isnull().sum()

Unnamed: 0               0
dischargePhase           0
voltage_charger          0
temperature_battery      0
current_load             0
time_end                 0
mode                     0
arrhenius_temperature    0
log_voltage              0
dtype: int64

### Cleaning - Second Life

In [49]:
second['arrhenius_temperature'] = 11605/(second['temperature_battery'] + 273.15)

#adding log_voltage column
second['log_voltage'] = np.log10(second['voltage_charger']) 

second['temp_volt'] = second['arrhenius_temperature'] * second['log_voltage']

second = second[second['temperature_battery'] > 0]

second = second[second['time_end'] > 10]

#display first few rows
second.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1393 entries, 0 to 1404
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             1393 non-null   int64  
 1   dischargePhase         1393 non-null   int64  
 2   voltage_charger        1393 non-null   float64
 3   temperature_battery    1393 non-null   float64
 4   current_load           1393 non-null   float64
 5   time_end               1393 non-null   float64
 6   mode                   1393 non-null   int64  
 7   arrhenius_temperature  1393 non-null   float64
 8   log_voltage            1393 non-null   float64
 9   temp_volt              1393 non-null   float64
dtypes: float64(7), int64(3)
memory usage: 119.7 KB


In [34]:
second.isnull().sum()

Unnamed: 0               0
dischargePhase           0
voltage_charger          0
temperature_battery      0
current_load             0
time_end                 0
mode                     0
arrhenius_temperature    0
log_voltage              0
dtype: int64

### Cleaning - Recomissioned

In [48]:
#adding arrhenius temperature column
recomissioned['arrhenius_temperature'] = 11605/(recomissioned['temperature_battery'] + 273.15)

#adding log voltage columns
recomissioned['log_voltage'] = np.log10(recomissioned['voltage_charger'])

recomissioned['temp_volt'] = recomissioned['arrhenius_temperature'] * recomissioned['log_voltage']

recomissioned = recomissioned[recomissioned['temperature_battery'] > 0]

recomissioned = recomissioned[recomissioned['time_end'] > 10]

#display first few rows
recomissioned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3534 entries, 0 to 3625
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             3534 non-null   int64  
 1   dischargePhase         3534 non-null   int64  
 2   voltage_charger        3534 non-null   float64
 3   temperature_battery    3534 non-null   float64
 4   current_load           3534 non-null   float64
 5   time_end               3534 non-null   float64
 6   mode                   3534 non-null   int64  
 7   arrhenius_temperature  3534 non-null   float64
 8   log_voltage            3534 non-null   float64
 9   temp_volt              3534 non-null   float64
dtypes: float64(7), int64(3)
memory usage: 303.7 KB


In [44]:
recomissioned.isnull().sum()

Unnamed: 0               0
dischargePhase           0
voltage_charger          0
temperature_battery      0
current_load             0
time_end                 0
mode                     0
arrhenius_temperature    0
log_voltage              0
temp_volt                0
dtype: int64

In [37]:
# Dropping one row with invalid log voltage
recomissioned = recomissioned.dropna(subset=['log_voltage'])

## Linear Regression

### Regular Batteries

In [55]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Splitting data into features and target
X = regular[['arrhenius_temperature', 'log_voltage', 'current_load','temp_volt']]
y = regular['time_end']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Linear regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Predictions
y_pred = lr_model.predict(X_test)

# Evaluation
#evaluations
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
n = len(y_test)
rss = np.sum((y_test-y_pred)**2)
k = X_train.shape[1] + 1
aic = n * np.log(rss / n) +2 * k
print(f'Mean Square Error: {mse}')
print(f'AIC Score: {aic}')
print(f'R^2 Score: {r2}')

Mean Square Error: 33084.62801462147
AIC Score: 7804.711208251066
R^2 Score: 0.9098979700916326


### Second Life Batteries

In [59]:
#splitting data into feature and target
X = second[['arrhenius_temperature', 'log_voltage' ,  'current_load' , 'temp_volt']]
y = second['time_end']

#train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#linear regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

#predictions
y_pred = lr_model.predict(X_test)

#evaluations
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
n = len(y_test)
rss = np.sum((y_test-y_pred)**2)
k = X_train.shape[1] + 1
aic = n * np.log(rss / n) +2 * k
print(f'Mean Square Error: {mse}')
print(f'AIC Score: {aic}')
print(f'R^2 Score: {r2}')

Mean Square Error: 7373.967396980429
AIC Score: 2494.6934129247543
R^2 Score: 0.960621393045031


### Recommissioned Batteries

In [56]:
#splitting data into feature and target
X = recomissioned[['arrhenius_temperature' , 'log_voltage' , 'current_load','temp_volt']]
y = recomissioned['time_end']

#train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#linear regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

#predictions
y_pred = lr_model.predict(X_test)

#evaluations
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
n = len(y_test)
rss = np.sum((y_test-y_pred)**2)
k = X_train.shape[1] + 1
aic = n * np.log(rss / n) +2 * k
print(f'Mean Square Error: {mse}')
print(f'AIC Score: {aic}')
print(f'R^2 Score: {r2}')

Mean Square Error: 22234.531893886804
AIC Score: 7086.647107987738
R^2 Score: 0.9306571035509208
